In [7]:
import re
import json
from loguru import logger
from tqdm import tqdm
import openai

client = openai.Client(base_url="http://127.0.0.1:30000/v1", api_key="EMPTY")

In [8]:
def get_prompt(problem, question, options):
    options = '\n'.join(f"{'ABCDEFG'[i]}. {o}" for i, o in enumerate(options))

    prompt = f"""你是一个逻辑推理专家，擅长解决逻辑推理问题。以下是一个逻辑推理的题目，形式为单项选择题。所有的问题都是（close-world assumption）闭世界假设，即未观测事实都为假。题目如下：
### 题目:
{problem}
### 问题:
{question}
{options}
"""
    return prompt + "请通过逐步推理来解答问题，答案只有1个，并把最终答案放置于\\boxed{}中"


In [9]:
def process_datas(datas):
    future_data = {}
    lens = 0
    for data in tqdm(datas, desc="Submitting tasks", total=len(datas)):
        problem = data['problem']
        messages = []
        for id, question in enumerate(data['questions']):
            if 'answer' in question and question['answer'] in 'ABCDEFG':
                continue
                
            prompt = get_prompt(problem,
                                question['question'],
                                question['options'],
                                )
            messages.append(prompt)
            lens += 1
        
        if len(messages) == 0:
            continue
        response = client.completions.create(
                            model="default",
                            prompt=messages,
                            temperature=0,
                            max_tokens=512
                        )
        
        for choice in response.choices:
            future_data[choice.text] = (data, choice.index)
   
    for future in tqdm(future_data, total=lens, desc="Processing tasks"):
        data = future_data[future][0]
        problem_id = future_data[future][1]
        try:
            answer_pattern = re.compile(r"boxed\{([A-Z])\}", re.S)
            answer = answer_pattern.findall(future)
            if len(answer) != 1 or answer[0] not in 'ABCDEFG':
                print(data)
                continue
                
            data['questions'][problem_id]['answer'] = answer[0]
        except Exception as e:
            logger.error(f"Failed to process text: {data}. Error: {e}")

    return datas

In [10]:
def all_has_answer(test_data):
    for item in test_data:
        for id, question in enumerate(item['questions']):
            if 'answer' not in question or question['answer'] not in 'ABCDEFG':
                return False
    return True

In [11]:
def main(ifn):
    problems = []
    # 按行读取数据
    with open(ifn) as reader:
        for line in reader:
            sample = json.loads(line)
            problems.append(sample)

    while not all_has_answer(problems):
        problems = process_datas(problems)
        print(problems)
    print("All tasks finished!")
    return problems

In [12]:
if __name__ == '__main__':
    return_list = main('round1_test_data.jsonl')

Processing tasks: 100%|██████████| 20/20 [00:00<00:00, 191958.99it/s]


{'problem': '**背景信息：**\n\n家谱信息如下：\n- 亚当是该隐、亚伯和塞特的父亲。\n- 塞特是恩诺斯的父亲。\n\n另外，我们在此家谱中定义：\n- 父亲的父亲称为祖父。\n- 有相同父亲的两个人是兄弟。\n- 祖父的儿子及其孙子之间的关系是伯、叔与侄的关系，其中伯、叔不是侄子的直接父亲。\n\n根据以上定义，回答下列选择题：', 'questions': [{'question': '选择题 1:\n恩诺斯的叔伯是谁？', 'options': ['亚当，塞特', '亚伯，该隐', '该隐，塞特', '塞特，亚当'], 'answer': 'A'}, {'question': '选择题 2:\n该隐是谁的叔伯？', 'options': ['亚当', '亚伯', '恩诺斯', '无']}, {'question': '选择题 3:\n该隐是否是恩诺斯的叔伯？', 'options': ['是', '否']}, {'question': '选择题 4:\n亚当是否是自己的叔伯？', 'options': ['是', '否']}], 'id': 'round1_test_data_009'}
{'problem': '在一个教育机构里，需要制定一周的工作计划，覆盖六天。有三位老师（Juan, Pedro, Ana）和他们的可用时间如下：\n- Juan：第1、3、4、6天可用。\n- Pedro：第1、3、5天可用。\n- Ana：第1、2、5、6天可用。\n\n需要确保每天至少有一位老师在工作，并且安排工作的老师在自己可用的时间内。每位老师在一周内，要工作相同数量的天数，并且天数和他们数量相同（即每个老师工作三天）。\n\n根据以上情况，请回答以下问题：', 'questions': [{'question': '选择题 1：\n如果合理安排Juan, Pedro, 和Ana的工作时间，以下那一种可能是他们一周的工作时间安排？', 'options': ['Juan: 第1、4、6天；Pedro: 第2、3、5天；Ana: 第1、5、6天', 'Pedro: 第1、3、5天；Ana: 第2、4、6天；Juan: 第1、3、4天', 'Pedro: 第1、3天；Ana: 第2、5、6天；Juan: 第4天', 'Pedro: 第1、3天；Ana

Processing tasks: 100%|██████████| 2/2 [00:00<00:00, 85598.04it/s]


[{'problem': '在一个火车网络中，存在以下直达火车连接：\n\n1. 从萨尔布吕肯到杜德维勒\n2. 从福尔巴到萨尔布吕肯\n3. 从弗雷明到福尔巴\n4. 从圣阿沃尔德到弗雷明\n5. 从法尔克蒙到圣阿沃尔德\n6. 从梅斯到法尔克蒙\n7. 从南锡到梅斯\n\n乘客可以直接乘坐火车或通过转乘到达目的地。基于上述信息，请回答以下问题：', 'questions': [{'question': '选择题 1：\n可以直接或间接从梅斯到弗雷明吗？', 'options': ['是', '否'], 'answer': 'A'}, {'question': '选择题 2：\n可以直接或间接从弗雷明到杜德维勒吗？', 'options': ['是', '否'], 'answer': 'A'}], 'id': 'round1_test_data_008'}, {'problem': '**背景信息：**\n\n家谱信息如下：\n- 亚当是该隐、亚伯和塞特的父亲。\n- 塞特是恩诺斯的父亲。\n\n另外，我们在此家谱中定义：\n- 父亲的父亲称为祖父。\n- 有相同父亲的两个人是兄弟。\n- 祖父的儿子及其孙子之间的关系是伯、叔与侄的关系，其中伯、叔不是侄子的直接父亲。\n\n根据以上定义，回答下列选择题：', 'questions': [{'question': '选择题 1:\n恩诺斯的叔伯是谁？', 'options': ['亚当，塞特', '亚伯，该隐', '该隐，塞特', '塞特，亚当'], 'answer': 'C'}, {'question': '选择题 2:\n该隐是谁的叔伯？', 'options': ['亚当', '亚伯', '恩诺斯', '无']}, {'question': '选择题 3:\n该隐是否是恩诺斯的叔伯？', 'options': ['是', '否'], 'answer': 'A'}, {'question': '选择题 4:\n亚当是否是自己的叔伯？', 'options': ['是', '否'], 'answer': 'B'}], 'id': 'round1_test_data_009'}, {'problem': '在一个教育机构里，需要制定一周的工作计划，覆盖六天。有三位老师（Juan, Pedro, Ana）和他们的可用

Processing tasks: 100%|██████████| 2/2 [00:00<00:00, 45100.04it/s]


...

[{'problem': '在一个火车网络中，存在以下直达火车连接：\n\n1. 从萨尔布吕肯到杜德维勒\n2. 从福尔巴到萨尔布吕肯\n3. 从弗雷明到福尔巴\n4. 从圣阿沃尔德到弗雷明\n5. 从法尔克蒙到圣阿沃尔德\n6. 从梅斯到法尔克蒙\n7. 从南锡到梅斯\n\n乘客可以直接乘坐火车或通过转乘到达目的地。基于上述信息，请回答以下问题：', 'questions': [{'question': '选择题 1：\n可以直接或间接从梅斯到弗雷明吗？', 'options': ['是', '否'], 'answer': 'A'}, {'question': '选择题 2：\n可以直接或间接从弗雷明到杜德维勒吗？', 'options': ['是', '否'], 'answer': 'A'}], 'id': 'round1_test_data_008'}, {'problem': '**背景信息：**\n\n家谱信息如下：\n- 亚当是该隐、亚伯和塞特的父亲。\n- 塞特是恩诺斯的父亲。\n\n另外，我们在此家谱中定义：\n- 父亲的父亲称为祖父。\n- 有相同父亲的两个人是兄弟。\n- 祖父的儿子及其孙子之间的关系是伯、叔与侄的关系，其中伯、叔不是侄子的直接父亲。\n\n根据以上定义，回答下列选择题：', 'questions': [{'question': '选择题 1:\n恩诺斯的叔伯是谁？', 'options': ['亚当，塞特', '亚伯，该隐', '该隐，塞特', '塞特，亚当'], 'answer': 'C'}, {'question': '选择题 2:\n该隐是谁的叔伯？', 'options': ['亚当', '亚伯', '恩诺斯', '无']}, {'question': '选择题 3:\n该隐是否是恩诺斯的叔伯？', 'options': ['是', '否'], 'answer': 'A'}, {'question': '选择题 4:\n亚当是否是自己的叔伯？', 'options': ['是', '否'], 'answer': 'B'}], 'id': 'round1_test_data_009'}, {'problem': '在一个教育机构里，需要制定一周的工作计划，覆盖六天。有三位老师（Juan, Pedro, Ana）和他们的可用

Processing tasks: 100%|██████████| 2/2 [00:00<00:00, 34807.50it/s]


{'problem': '在一个教育机构里，需要制定一周的工作计划，覆盖六天。有三位老师（Juan, Pedro, Ana）和他们的可用时间如下：\n- Juan：第1、3、4、6天可用。\n- Pedro：第1、3、5天可用。\n- Ana：第1、2、5、6天可用。\n\n需要确保每天至少有一位老师在工作，并且安排工作的老师在自己可用的时间内。每位老师在一周内，要工作相同数量的天数，并且天数和他们数量相同（即每个老师工作三天）。\n\n根据以上情况，请回答以下问题：', 'questions': [{'question': '选择题 1：\n如果合理安排Juan, Pedro, 和Ana的工作时间，以下那一种可能是他们一周的工作时间安排？', 'options': ['Juan: 第1、4、6天；Pedro: 第2、3、5天；Ana: 第1、5、6天', 'Pedro: 第1、3、5天；Ana: 第2、4、6天；Juan: 第1、3、4天', 'Pedro: 第1、3天；Ana: 第2、5、6天；Juan: 第4天', 'Pedro: 第1、3天；Ana: 第2、5、6天；Juan: 第4、5、6天'], 'answer': 'A'}, {'question': '选择题 2：\n选择一种合理的工作安排方式为Juan, Pedro, 和Ana, 假设每位老师的工作天数一致。', 'options': ['Juan: 第1、4、6天；Pedro: 第2、3、5天；Ana: 第1、2、6天', 'Pedro: 第1、3、5天；Ana: 第2、4、6天；Juan: 第1、2、4天', 'Pedro: 第1、3天；Ana: 第2、5、6天；Juan: 第4、5、6天', 'Juan: 第1、3、4天；Pedro: 第1、3、5天；Ana: 第2、5、6天']}], 'id': 'round1_test_data_010'}
[{'problem': '在一个火车网络中，存在以下直达火车连接：\n\n1. 从萨尔布吕肯到杜德维勒\n2. 从福尔巴到萨尔布吕肯\n3. 从弗雷明到福尔巴\n4. 从圣阿沃尔德到弗雷明\n5. 从法尔克蒙到圣阿沃尔德\n6. 从梅斯到法尔克蒙\n7. 从南锡到梅斯\n\n乘客可以直接乘坐火车或通过转乘到达目的地。基于上述信息，请回答以下问题：',

Submitting tasks:  22%|██▏       | 2/9 [00:04<00:16,  2.30s/it]


KeyboardInterrupt: 

In [ ]:
sorted_data = sorted(return_list, key=lambda x: int(str(x['id'])[-3:]))
with open('upload-pipeline.jsonl', 'w') as writer:
    for sample in sorted_data:
        writer.write(json.dumps(sample, ensure_ascii=False))
        writer.write('\n')